In [1]:
using Dates: Date, DateTime;
using Printf
using JuMP

In [2]:
PTDF_TRIMMER = 1e-6;

root_path = @__DIR__;
push!(LOAD_PATH, root_path);
cd(root_path);

include(joinpath(root_path, "scopf_utils.jl"))
include(joinpath(root_path, "AmplTxt.jl"));
include(joinpath(root_path, "SCOPF.jl"));
include(joinpath(root_path, "Workflow.jl"));

optimizer: Cbc.Optimizer


# 2-buses

In [ ]:
base_data_name = "data/data_slack/2buses/base";

## base PTDF with bus 1 as reference

In [3]:
amplTxt = AmplTxt.read(base_data_name);
base_data_path = joinpath(root_path, base_data_name)
network = SCOPF.Network(amplTxt);

ref_bus = 1;
B = SCOPF.get_B(network, 1e-6);
Binv = SCOPF.get_B_inv(B, ref_bus);
PTDF = SCOPF.get_PTDF(network, Binv, ref_bus);

file_path_l =  joinpath(base_data_path, "ptdf.txt")
SCOPF.write_PTDF(file_path_l, network, ref_bus, PTDF, PTDF_TRIMMER)

n = 2
ref_bus is 1
n 2
m 1


## generate multiple coeffs distribution for PTDFs calculation

In [4]:
nb_buses_l = size(PTDF)[2]
dict_coeffs = Dict( "uniform" => [0.5, 0.5],
                    "0_1" => [0., 1.],
                    "1_0" => [1., 0.])
SCOPFutils.pretty_print(dict_coeffs, 3)

   1_0 => [1.0, 0.0]
   0_1 => [0.0, 1.0]
   uniform => [0.5, 0.5]


## launch PSCOPF with different PTDFs

In [8]:
ECH = DateTime("2015-01-01T11:00:00")
P_RES_MIN = 0; #use positive value
P_RES_MAX = 500;
@assert((P_RES_MIN>=0) & (P_RES_MAX>=0) & (P_RES_MIN<=P_RES_MAX))

TS = DateTime("2015-01-01T11:00:00")
SCENARIO = "S1"

#dict_flows[reserve][branch][testname] = flow 
dict_flows = Dict{Float64, Dict{String, Dict{String, Float64}}}()
dict_flows[P_RES_MAX] = Dict{String, Dict{String, Float64}}()
#get!(dict_flows, 105., Dict{String, Float64}())["az"] = 15.
dict_prod = Dict{Float64, Dict{String, Dict{String, Float64}}}()
dict_prod[P_RES_MAX] = Dict{String, Dict{String, Float64}}()

for (test_name_l, coeffs_l) in dict_coeffs
    usecase_name_l = @sprintf("%s_RES_%s_%s", test_name_l, P_RES_MIN, P_RES_MAX)
    usecase_path_l = joinpath(dirname(rstrip(base_data_path,'/')), usecase_name_l)
    mkdir(usecase_path_l)
    PTDF_distributed = SCOPF.distribute_slack(PTDF, 1, coeffs_l);
    
    println(file_path_l, " :", coeffs_l)
    file_path_l = joinpath(usecase_path_l, "pscopf_ptdf.txt")
    SCOPF.write_PTDF(file_path_l, network, ref_bus, PTDF_distributed, PTDF_TRIMMER)
    file_path_l = joinpath(usecase_path_l, "coeffs.txt")
    SCOPF.write_slack_distribution(file_path_l, network, coeffs_l)
    
    #read input from "base" folder but write into the "test" folder
    launcher_l = Workflow.Launcher(base_data_path, usecase_path_l);
    
    launcher_l.ptdf = Workflow.read_ptdf(usecase_path_l)
    
    model, v_lim, v_imp, v_res, v_flow = Workflow.sc_opf(launcher_l, ECH, -P_RES_MIN, P_RES_MAX);
    for ((branch_l, _, s), flow_var_l) in v_flow
        get!(dict_flows[P_RES_MAX], branch_l, Dict{String, Float64}());
        if s == SCENARIO
            dict_flows[P_RES_MAX][branch_l][test_name_l] = value(flow_var_l)
        end
    end
    
    #dict_prod[P_RES_MAX][unit_l][test_name_l]
end

dict_flows

/home/aziz/Projects/RTE/PSCOPF/pscopf/data/data_slack/2buses/uniform_RES0_500/coeffs.txt :[1.0, 0.0]
NAMES : Set(["poste_1_0", "wind_1", "sundance", "alta", "poste_2_0"])
BUSES : Set(["poste_1_0", "poste_2_0"])
units_by_bus : Dict("Imposable" => Dict("poste_1_0" => ["alta"], "poste_2_0" => ["sundance"]), "Limitable" => Dict("poste_1_0" => ["wind_1"], "poste_2_0" => []))
Number of scenario 1
Number of time step is 1
[DateTime("2015-01-01T11:00:00")]
eod_slack is Dict((DateTime("2015-01-01T11:00:00"), "S1") => 0.0)
alta 2015-01-01T11:00:00 is at 0.000000

sundance 2015-01-01T11:00:00 is at 0.000000

ptdf_by_branch : Dict("1_2" => Dict("poste_2_0" => -1.0))
end of optim.
OPTIMAL
300.00010000000003
NO_IMPOSABLE   : false
NO_LIMITABLE   : false
NO_LIMITATION  : false
BUSES          : Set(["poste_1_0", "poste_2_0"])
NAMES          : Set(["poste_1_0", "wind_1", "sundance", "alta", "poste_2_0"])
TS             : [DateTime("2015-01-01T11:00:00")]
S              : ["S1"]
                        

Dict{Float64, Dict{String, Dict{String, Float64}}} with 1 entry:
  500.0 => Dict("1_2"=>Dict("1_0"=>100.0, "0_1"=>100.0, "uniform"=>100.0))

In [ ]:
SCOPFutils.pretty_print(dict_flows)

In [ ]:
    for (kind_l, dict_units) in Workflow.get_units_by_kind(launcher_l)
        for unit_l, _ in dict_units
            get!(dict_prod[P_RES], unit_l, Dict{String, Float64}());
            if kind_l == Workflow.K_IMPOSABLE
                dict_prod[P_RES][unit_l][test_name_l] = value(v_imp.p_imposable[unit_l, TS, SCENARIO])
            elseif kind_l == Workflow.K_LIMITABLE
                dict_prod[P_RES][unit_l][test_name_l] = value(v_imp.p_enr[unit_l, TS, SCENARIO])
            end
        end
    end